In [1]:
import pandas as pd
import re
import emoji
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import MinMaxScaler
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch

In [2]:
df1 = pd.read_csv('../raw_data/twitter_1.csv')
df2 = pd.read_csv('../raw_data/twitter_2.csv')
df = pd.concat([df1, df2], ignore_index=True)

/var/folders/sd/p4tcs9gs0hn4nt45wd9f9q1w0000gn/T/ipykernel_13931/399126454.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('../raw_data/twitter_2.csv')


In [3]:
display(df1.shape)
display(df2.shape)

(139488, 17)

(229584, 17)

In [4]:
df.rename(columns={'rawContent': 'tweet'}, inplace=True)

In [5]:
def convert_to_datetime(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    return df

In [6]:
def convert_emojis(string):
    text = emoji.demojize(string)
    text = text.replace(":", "").replace("_", " ")
    return text

In [7]:
def clean_tweet(tweet):
    
    # Remove url
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)
    tweet = re.sub(r'http?:\/\/\S+', '', tweet)
    
    # Remove \n
    tweet = re.sub(r'\n', '', tweet)
    
    # Remove @
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)
    
    # Remove #
    tweet = re.sub(r'#', '', tweet)
    
    # Remove RT
    tweet = re.sub(r'RT[\s]+', '', tweet)
    
    # Convert to lower case
    tweet = tweet.lower()
    
    # Remove "#&'()*/:;@[\]^`{|}~"
    tweet = re.sub(r'[#&\'\(\)\*\+\/:;@\[\]\^`{|}~]', '', tweet)
    
    # Remove double space
    tweet = re.sub(r'\s\s+', ' ', tweet)
    
    return tweet

In [8]:
df = convert_to_datetime(df)
df['tweet'] = df['tweet'].apply(clean_tweet)
df['tweet'] = df['tweet'].apply(convert_emojis)

In [9]:
list1 = ['AAPL', 'Apple Inc.', 'Appl', 
         'AMC', 'AMC Entertainment Holdings Inc.', 'AAMC', 
         'AC', 'AMZN', 'Amazon.com Inc.', 'Amazn', 'AMD', 
         'Advanced Micro Devices Inc.', 'AMED', 
         'BB', 'BlackBerry Ltd.', 'Blacberry', 
         'BBBY', 'Bed Bath & Beyond Inc.', 'Bedbathbeyond', 
         'BTC', 'Bitcoin', 'bit coin', 'BYND', 'Beyond Meat Inc.', 'Bynd', 
         'CLNE', 'Clean Energy Fuels Corp.', 'Clnenergy', 
         'CMG', 'Chipotle Mexican Grill Inc.', 'Chipotle', 'COST', 
         'Costco Wholesale Corp.', 'Coscto', 'CRSR', 'Corsair Gaming Inc.', 
         'Corsair', 'DIS', 'Walt Disney Co.', 'Disney', 'DOGE', 'Dogecoin', 
         'Doge coin', 'ET', 'Energy Transfer LP', 'Energy Transfer', 'F', 
         'Ford Motor Co.', 'Ford', 'FB', 'Facebook Inc.', 'Fb', 'GME', 
         'GameStop Corp.', 'Gamestop', 'GOOG', 'Alphabet Inc.', 'Google', 
         'HD', 'Home Depot Inc.', 'Home Depo', 'INTC', 'Intel Corp.', 
         'Intell', 'JNJ', 'Johnson & Johnson', 'Johnsohn', 'KO', 
         'Coca-Cola Co.', 'Coca Cola', 'LULU', 'Lululemon Athletica Inc.', 
         'Lululemn', 'MCD', "McDonald's Corp.", "McDonalds", 'MGM', 
         'MGM Resorts International', 'Mgmresorts', 'MSFT', 'Microsoft Corp.', 
         'Micrsoft', 'MU', 'Micron Technology Inc.', 'Micron', 
         'NIO', 'NIO Inc.', 'Nio', 'NVDA', 'NVIDIA Corp.', 'Nivida', 
         'PFE', 'Pfizer Inc.', 'Pifzer', 'PINS', 'Pinterest Inc.', 
         'Pintrst', 'PLTR', 'Palantir Technologies Inc.', 'Palintir', 
         'QQQ', 'Invesco QQQ Trust', 'InvescoQQQ', 'RBLX', 
         'Roblox Corp.', 'Robloks', 'RIOT', 'Riot Blockchain Inc.', 
         'Riott', 'ROKU', 'Roku Inc.', 'Rokue', 'SNDL', 
         'Sundial Growers Inc.', 'Sundail', 'SPCE', 
         'Virgin Galactic Holdings Inc.', 'Spacex', 'SQ', 'Square Inc.', 
         'Squar', 'T', 'AT&T Inc.', 'Att', 'TSLA', 'Tesla Inc.', 
         'Teslla', 'TWTR', 'Twitter Inc.', 'Twiter', 'UBER', 
         'Uber Technologies Inc.', 'Ube', 'UPST', 'Upstart Holdings Inc.', 
         'Upstartholdings', 'V', 'Visa Inc.', 'Visa', 'WMT', 'Walmart Inc.', 
         'Walmrt', 'XOM', 'Exxon Mobil Corp.', 'Exxonmobil']
list2 = ['AAL', 'American Airlines Group Inc.', 'Americanairlines',
        'ABNB', 'Airbnb Inc.', 'AirBnB',
        'ACB', 'Aurora Cannabis Inc.', 'Auroracannabis',
        'AMRN', 'Amarin Corp. plc', 'Amerin',
        'ARKK', 'ARK Innovation ETF', 'ARKinovation',
        'BABA', 'Alibaba Group Holding Ltd.', 'Alibaba',
        'BA', 'Boeing Co.', 'Boing',
        'BAC', 'Bank of America Corp.', 'Bankofamerica',
        'BIDU', 'Baidu Inc.', 'Bido',
        'BILI', 'Bilibili Inc.', 'Billibili',
        'BLNK', 'Blink Charging Co.', 'Blinkcharg',
        'BMY', 'Bristol Myers Squibb Co.', 'Bristolmyers',
        'BRK.A', 'Berkshire Hathaway Inc.', 'Berkshira',
        'CCL', 'Carnival Corp.', 'Carnival',
        'CGC', 'Canopy Growth Corp.', 'Canopygrowth',
        'CHWY', 'Chewy Inc.', 'Chewycom',
        'CSCO', 'Cisco Systems Inc.', 'Cicsco',
        'CVS', 'CVS Health Corp.', 'Cvshealth',
        'DAL', 'Delta Air Lines Inc.', 'Deltaairlines',
        'DDOG', 'Datadog Inc.', 'DataDog',
        'DISCA', 'Discovery Inc. - Class A', 'Discovera',
        'DKNG', 'DraftKings Inc.', 'DraftKings',
        'ENPH', 'Enphase Energy Inc.', 'Enphase',
        'EQT', 'EQT Corp.', 'Eqtcorporation',
        'FCEL', 'FuelCell Energy Inc.', 'Fuelcell',
        'FSLY', 'Fastly Inc.', 'Fastly',
        'GE', 'General Electric Co.', 'GeneralElectric',
        'GM', 'General Motors Co.', 'GeneralMotors',
        'GOLD', 'Barrick Gold Corp.', 'Goldmining',
        'GPRO', 'GoPro Inc.', 'Gopro',
        'GRWG', 'GrowGeneration Corp.', 'Growgen',
        'HPE', 'Hewlett Packard Enterprise Co.', 'Hewlettpackard',
        'IBB', 'iShares NASDAQ Biotechnology ETF', 'iSharesNasdaqBiotechnology',
        'IBKR', 'Interactive Brokers Group Inc.', 'InterectiveBrokers',
        'INO', 'Inovio Pharmaceuticals Inc.', 'Inovio',
        'JD', 'JD.com Inc.', 'JD',
        'JMIA', 'Jumia Technologies AG', 'Jumia',
        'JPM', 'JPMorgan Chase & Co.', 'JPmorgan',
        'KHC', 'Kraft Heinz Co.', 'KraftHeinz',
        'LIT', 'Global X Lithium & Battery Tech ETF', 'GlobalXlithium',
        'MARA', 'Marathon Digital Holdings Inc.', 'MarathonDigital',
        'MRNA', 'Moderna Inc.', 'Modernna',
        'NCLH', 'Norwegian Cruise Line Holdings Ltd.', 'Norwegiancruiseline',
        'NET', 'Cloudflare Inc.', 'CloudFlare',
        'NFLX', 'Netflix Inc.', 'Netlfix',
        'NKE', 'Nike Inc.', 'Nike', 'NKLA', 'Nikola Corp.', 'Nicola', 'NOK', 'Nokia Corp.', 'Noka', 'O', 'Realty Income Corp.', 'Realtyincome', 'OGI', 'OrganiGram Holdings Inc.', 'Organigram', 'OTRK', 'Ontrak Inc.', 'OnTrack', 'PDD', 'Pinduoduo Inc.', 'Pinduoduo', 'PENN', 'Penn National Gaming Inc.', 'PennNational']


In [10]:
keywords = []

for word in list1:
    word = clean_tweet(word)
    word = word.replace('inc.', '').replace('ltd.', '').replace('co.', '').replace('corp.', '').replace('.com', '')
    keywords.append(word)

for word in list2:
    word = clean_tweet(word)
    word = word.replace('inc.', '').replace('ltd.', '').replace('co.', '').replace('corp.', '').replace('.com', '')
    keywords.append(word)

In [11]:
top50 = pd.read_csv('../raw_data/top50.csv')
cashtags = top50['cashtags'].tolist()

In [12]:
for cashtag in cashtags:
    cashtag = clean_tweet(cashtag)
    cashtag = re.sub(r'[^\w\s]', '', cashtag)
    cashtag = cashtag.replace(' ', '')
    
    if cashtag not in keywords:
        keywords.append(cashtag)

In [13]:
for keyword in keywords:
    if len(keyword) < 2:
        keywords.remove(keyword)

In [14]:
df.shape

(369072, 17)

In [23]:
condition1 = df['cashtags'].notnull()
condition2 = df['hashtags'].notnull()
condition3 = df['tweet'].str.contains(keyword, case=False)

In [24]:
# Filter the DataFrame based on the conditions
final_data = df[condition1 | condition2 | condition3]

# Reset the index of the filtered DataFrame
final_data = final_data.reset_index(drop=True)

In [25]:
final_data.shape

(69137, 17)

In [27]:
final_data.to_csv('../raw_data/final_data.csv', index=False)